In [1]:
import numpy as np
import tensorflow as tf

from keras import layers
from keras.layers import Input,Add,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

import pydot
from IPython.display import SVG
import scipy.misc
import kt_utils
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import resnets_utils
% matplotlib inline

Using TensorFlow backend.


![avatar](https://img-blog.csdn.net/20180509112628635?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTM3MzMzMjY=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

### 恒等块

In [4]:
def identity_block(X,f,filters,stage,block):
    '''
    如上图，实现卷积块
    
    参数：
        X - 输入的tensor类型的变量，维度为(m,n_H_prev,n_W_prev,n_C_prev)
        f - 整数，指定主路径中间的Conv窗口的维度
        filters - 整数l列表，定义主路径每层的卷积层的过滤器数量
        stage - 整数，根据每层的位置来命名每一层，与block参数一起使用
        block - 字符串，根据每层的位置来命名每一层，与stage参数一起使用
        
    返回:
        X - 卷积块的输出，tensor类型的变量，维度为(n_H,n_W,n_C)
    '''
    # 定义命名规则
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # 获取过滤器数量
    F1,F2,F3 = filters
    #保存输入数据
    X_short_cut = X
    
    #主路径
    ## 第一部分
    X = Conv2D(filters = F1,kernel_size = (1,1),strides=(1,1),padding='valid',
              name = conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    ## 第二部分
    X = Conv2D(filters = F2,kernel_size = (f,f),strides=(1,1),padding='same',
              name = conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    ## 第三部分
    X = Conv2D(filters = F3,kernel_size = (1,1),strides=(1,1),padding='valid',
              name = conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    
    ## 捷径
    ## 最后一步
    X = Add()([X,X_short_cut])
    X = Activation('relu')(X)
    
    return X 

In [5]:
tf.reset_default_graph()
import numpy as np
with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder(tf.float32,[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    A = identity_block(A_prev,f=2,filters=[2,4,6],stage=1,block='a')
    
    init = tf.global_variables_initializer()
    test.run(init)
    out = test.run([A],feed_dict = {A_prev:X,K.learning_phase():0})
    print('out='+str(out[0][1][1][0]))
    test.close()

out=[0.94822985 0.         1.1610144  2.747859   0.         1.36677   ]


![avatar](https://img-blog.csdn.net/20180509112838777?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTM3MzMzMjY=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

## 卷积块

In [6]:
def convolutional_block(X,f,filters,stage,block,s=2):
    '''
    如上图，实现卷积块
    
    参数：
        X - 输入的tensor类型的变量，维度为(m,n_H_prev,n_W_prev,n_C_prev)
        f - 整数，指定主路径中间的Conv窗口的维度
        filters - 整数l列表，定义主路径每层的卷积层的过滤器数量
        stage - 整数，根据每层的位置来命名每一层，与block参数一起使用
        block - 字符串，根据每层的位置来命名每一层，与stage参数一起使用
        s - 整数，指定要使用的步幅
        
    返回:
        X - 卷积块的输出，tensor类型的变量，维度为(n_H,n_W,n_C)
    '''
    # 定义命名规则
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # 获取过滤器数量
    F1,F2,F3 = filters
    #保存输入数据
    X_short_cut = X
    
    #主路径
    ## 第一部分
    X = Conv2D(filters = F1,kernel_size = (1,1),strides=(s,s),padding='valid',
              name = conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    ## 第二部分
    X = Conv2D(filters = F2,kernel_size = (f,f),strides=(1,1),padding='same',
              name = conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    ## 第三部分
    X = Conv2D(filters = F3,kernel_size = (1,1),strides=(1,1),padding='valid',
              name = conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    
    ## 捷径
    X_short_cut = Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),padding='valid',
                        name=conv_name_base+'1',kernel_initializer=glorot_uniform(seed=0))(X_short_cut)
    X_short_cut = BatchNormalization(axis=3,name=bn_name_base+'1')(X_short_cut)
    ## 最后一步
    X = Add()([X,X_short_cut])
    X = Activation('relu')(X)
    
    return X 

In [7]:
tf.reset_default_graph()
import numpy as np
with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder('float',[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    
    A = convolutional_block(A_prev,f=2,filters=[2,4,6],stage=1,block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print('out='+str(out[0][1][1][0]))
    
    test.close()

out=[0.09018463 1.2348977  0.46822017 0.0367176  0.         0.65516603]


![avatar](https://img-blog.csdn.net/20180509113317410?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTM3MzMzMjY=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

In [8]:
def ResNet50(input_shape=(64,64,3),classes=6):
    '''
    实现50层的残差网络
    CONV2D->BATCHNORM->RELU->MAXPOOL->CONVBLOCK->IDBLOCK*2->CONVBLOCK->IDBLOCK*3
    ->CONVBLOCK->IDBLOCK*5->CONVBLOCK->IDBLOCK*2->AVGPOOL->TOPLAYER
    
    参数：
        input_shape - 图像数据集的维度
        classes - 整数，分类数
    
    返回:
        model -Keras的框架
    '''
    
    X_input = Input(input_shape)
    # 0 填充
    X = ZeroPadding2D((3,3))(X_input)
    #stage 1
    X = Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides=(2,2))(X)
    
    # stage 2
    X = convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
    X = identity_block(X,f=3,filters=[64,64,256],stage=2,block='b')
    X = identity_block(X,f=3,filters=[64,64,256],stage=2,block='c')

    # stage 3
    X = convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='b')
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='c')    
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='d')    
    
    # stage 4
    X = convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block='b')    
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block='c')        
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block='d')        
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block='e')   
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block='f')    
    
    # stage 5
    X = convolutional_block(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
    X = identity_block(X,f=3,filters=[512,512,2048],stage=5,block='b')   
    X = identity_block(X,f=3,filters=[512,512,2048],stage=5,block='c')    
    
    # 均值池化层
    X = AveragePooling2D(pool_size=(2,2),padding='same')(X)
    
    # 输出层
    X = Flatten()(X)
    X = Dense(classes,activation='softmax',name='fc'+str(classes),
             kernel_initializer=glorot_uniform(seed=0))(X)
    
    # 创建模型
    model = Model(inputs=X_input,outputs=X,name='ResNet50')
    
    return model

In [9]:
model = ResNet50(input_shape=(64,64,3),classes=6)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
## 加载模型
X_train_orig,Y_train_orig,X_test_orig,Y_test_orig,classes = resnets_utils.load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = resnets_utils.convert_to_one_hot(Y_train_orig, 6).T
Y_test = resnets_utils.convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [11]:
# 训练模型
model.fit(X_train,Y_train,epochs=2,batch_size=32)

Epoch 1/2
1080/1080 [==============================] - 241s - loss: 2.9342 - acc: 0.2528   
Epoch 2/2
1080/1080 [==============================] - 232s - loss: 2.0170 - acc: 0.3213   


In [13]:
pred = model.evaluate(X_test,Y_test)

print('误差值='+str(pred[0]))
print('准确率='+str(pred[1]))

120/120 [==============================] - 6s     
误差值=2.0816587448120116
准确率=0.16666666666666666


In [14]:
from PIL import image
import numpy as np
import matplotlib.pyplot as plt

img_path = ''

my_image = image.load_img(img_path,size=(64,64))
my_image = image.img_to_array(my_image)

my_image = np.expand_dims(my_image,axis=0)
my_image = preprocess_input(my_image)

print('my_image.shape = 'str(my_image.shape))
print('class prediction vector [p(0),p(1),p(2),p(3),p(4),p(5)]=')
print(model.predict(my_image))

my_image = scipy.misc.imread(img_path)
plt.imshow(my_image)

SyntaxError: invalid syntax (<ipython-input-14-0c0ab2429450>, line 13)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64, 64, 3)     0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 70, 70, 3)     0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 32, 32, 64)    9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 32, 32, 64)    256         conv1[0][0]                      
___________________________________________________________________________________________